<a href="https://colab.research.google.com/github/arnauldnzegha/deep2pde_Berg_Nystrom/blob/master/Berg_Nystrom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras import backend as K 
from keras.models import Model
import math as M
from keras.optimizers import SGD,Adadelta
from keras.layers import Dense, Input
import numpy as np
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
import scipy.io
from keras.regularizers import l1
from random import shuffle

*** burgers_sine.mat*** : https://drive.google.com/file/d/1NOYWXhdm6i3zyU5nbcteS617lCDKyc7P/view?usp=sharing

Download the file on your local drive then execute the next code to upload it on colab.
The file content a compiled matrix of data

In [0]:
from google.colab import files
#files.upload()

the **getBurgersData( test  )** function retrieves the data in the **burgers_sine.mat** file and puts it in an array [x_values,  time_values] and an array [U_value].
These datas in *** burgers_sine.mat file***  are generated by a solution of the burgers equation.

In [0]:

#*******************GENERATE BURGERS *******************************************
def getBurgersData(test=0.07):
    data_sol = scipy.io.loadmat('burgers_sine.mat')  
    t_sol = data_sol['t'].flatten()[:,None]
    x_sol = data_sol['x'].flatten()[:,None]
    usol = data_sol['usol'].flatten()[:,None]
    Exact_sol = np.real(data_sol['usol'])
    T_sol, X_sol = np.meshgrid(t_sol,x_sol)
    t_sol_star = T_sol.flatten()[:,None]
    x_sol_star = X_sol.flatten()[:,None]
    X_sol_star = np.hstack((t_sol_star, x_sol_star))
    u_sol_star = Exact_sol.flatten()[:,None]
    n_train = int(len(u_sol_star)*(1-test))
    X_sol_star, u_sol_star= mixData(X_sol_star, u_sol_star)
    X_train, Y_train = X_sol_star[:n_train, :],  u_sol_star[:n_train]
    X_test, Y_test = X_sol_star[n_train:, :], u_sol_star[n_train:]
    return  X_train, Y_train, X_test, Y_test
#*******************END GENERATE BURGERS *******************************************

The next code mix data in a random order.
It return the same array but in a  random order.

In [0]:
#******************* MIX DATA *******************************************
def mixData(xs,ys):
    xys=[]
    for i in range(0,len(xs)-1):
        xys.append((xs[i],ys[i]))
    shuffle(xys)
    x2,y2=[],[]
    for (x,y) in xys:
        x2.append(x)
        y2.append(y)
    return (np.asarray(x2), np.asarray(y2))
#*******************END MIX DATA *******************************************

The first model is a neural network (a MLP) with **5 hidden** layers and **10 units by hidden** layers.

The activation is hyperbolic tangent.
L1 regulation terme is used during de training

In [0]:
#******************* FIRST MODEL***********************************************
def my_model(nb_class=1):
    l1_Reg = l1(0.001)
    input_data = Input(shape=(2,))
    x = Dense(10, activation='tanh', name='l1' ,kernel_regularizer=l1_Reg)(input_data)
    x = Dense(10, activation='tanh', name='l2' ,kernel_regularizer=l1_Reg)(x)
    x = Dense(10, activation='tanh', name='l3' ,kernel_regularizer=l1_Reg)(x)
    x = Dense(10, activation='tanh', name='l4' ,kernel_regularizer=l1_Reg)(x)
    x = Dense(10, activation='tanh', name='l5' ,kernel_regularizer=l1_Reg)(x)
    x = Dense(nb_class, activation='linear', name='sort' ,kernel_regularizer=l1_Reg)(x)
    model = Model(input_data, x)
    return model
#*******************END MODEL*****************************************************************

In [0]:


#******************* MODEL 2 EDP***********************************************
"""  Reseau 2 celui sous forme d'EDP """
def modelEDP(nb_class=1):
    input_data = Input(shape=(3,))
    x = Dense(2, activation='tanh', name='l1')(input_data)
    x = Dense(2, activation='tanh', name='l2')(x)
    x = Dense(nb_class, activation='linear', name='sort')(x)
    model = Model(input_data, x)
    return model
#*******************END MODEL 2 EDP*****************************************************************


**dataTrainModelEDP(X,U)** 

take a liste(** [x_value , time_val]** and** [u_value],** )  

and return a liste   **([U_value, x derivative of U, second order x derivative of U]**  and **[time derivative of U named Ut]**)

In [0]:

def dataTrainModelEDP(X,U):
    X_der=np.zeros(len(X))
    T_der=np.zeros(len(X))
    X_der2=np.zeros(len(X))
    X_train=np.zeros([len(X),3])
    for i in range(0, len(X)):
        layer_outs = get_layer_outputs(X[i])
        X_train[i][0]=U[i]
        X_train[i][1]=derivePartiel(-1,0,X[i],layer_outs,tanh_deriv)
        X_train[i][2]=derivePartiel(-1,1,X[i],layer_outs,tanh_deriv)
        X_der2[i]=derivePartiel(-1,0,X[i],layer_outs, tanh_deriv2)
    return X_train, X_der2

This code returne in a list the output of each layer after a prediction.

In [0]:
def get_layer_outputs(test_data):
    test_data=test_data.reshape(1,2,)
    ar=np.float32(test_data)
    test_data2=[[ar]]
    outputs = [layer.output for layer in model.layers[1:]]
    functors = [K.function([model.input, K.learning_phase()], [out]) for out in outputs] 
    layer_outs = [func([test_data, 1.]) for func in functors]
    return test_data2+layer_outs

**tanh_deriv** return the derivative of tanh. **th' = (1-th^2)**


**tanh_deriv2** return the second order derivative **th' ' =  -2(th - th^3)**


In [0]:
#******************* DERIVEE  *******************************************
def tanh_deriv(x):
    return(1-(M.tanh(x))**2)

def tanh_deriv2 (x):
    return(-2*(M.tanh(x) -(M.tanh(x))**3))
#*******************END DERIVEE  *******************************************

In [0]:
#******************* DERIVATION 2***********************************************
def derivePartiel(layer, n_i, inputX, layer_outs, fct_deriv):
    nb_l=len(model.layers)
    if((layer)>(-1*nb_l)):
        ln=len(model.layers[layer].get_weights()[0][0])
        if((layer-1)>(-1*nb_l)):
            inp=len(model.layers[layer-1].get_weights()[0][0])
        else:
            inp=len(inputX)
        Lpartial=np.zeros(ln)
        LpartialN=derivePartiel(layer-1 ,n_i, inputX,layer_outs, fct_deriv)
        chaine=0
        for n in range(0,ln):
            act=model.layers[layer].get_weights()[1][n]
            chaine=0
            for i in range(0,inp):
                W=model.layers[layer].get_weights()[0][i][n]
                act+=W*layer_outs[layer-1][0][0][i]
                chaine+=W*LpartialN[i]
            if (layer==-1):
                Lpartial[n]=chaine
            else:
                Lpartial[n] = fct_deriv(act)*chaine
        return Lpartial
    else:
        Lpartial=np.zeros(len(inputX))
        Lpartial[n_i]=1
        return Lpartial
#*******************END DERIVATION*****************************************************************


The training phase with SDG and the mean squared error.


In [74]:
#*******************MAIN*************************************************************************
if __name__ == '__main__':
    model = my_model()
    model2 = modelEDP()
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    """ model compilation """
    model.compile(loss='mean_squared_error', optimizer=sgd,  metrics=['accuracy'])
    model2.compile(loss='mean_squared_error', optimizer=sgd,  metrics=['accuracy'])

    print("train M1")
    X_train, Y_train, X_test, Y_test= getBurgersData()
    hist=model.fit(X_train, Y_train, validation_split=0.15, batch_size=500, epochs=1000, verbose=1)

    print("calculate derivatives") 
    X_train2, Y_train2=X_train[:10, :], Y_train[:10,]
    Uxx, Ut=dataTrainModelEDP(X_train2, Y_train2)
    print("train M2")
    hist=model2.fit(Uxx, Ut, validation_split=0.1, batch_size=1000, epochs=5, verbose=1)

train M1
Train on 40675 samples, validate on 7179 samples
Epoch 1/1000
40675/40675 [==============================] - 1s 19us/step - loss: 0.1881 - acc: 4.9170e-05 - val_loss: 0.1336 - val_acc: 1.3930e-04
Epoch 2/1000
40675/40675 [==============================] - 0s 8us/step - loss: 0.1279 - acc: 4.9170e-05 - val_loss: 0.1257 - val_acc: 1.3930e-04
Epoch 3/1000
40675/40675 [==============================] - 0s 8us/step - loss: 0.1226 - acc: 4.9170e-05 - val_loss: 0.1214 - val_acc: 1.3930e-04
Epoch 4/1000
40675/40675 [==============================] - 0s 7us/step - loss: 0.1185 - acc: 4.9170e-05 - val_loss: 0.1175 - val_acc: 1.3930e-04
Epoch 5/1000
40675/40675 [==============================] - 0s 8us/step - loss: 0.1148 - acc: 4.9170e-05 - val_loss: 0.1151 - val_acc: 1.3930e-04
Epoch 6/1000
40675/40675 [==============================] - 0s 7us/step - loss: 0.1119 - acc: 4.9170e-05 - val_loss: 0.1106 - val_acc: 1.3930e-04
Epoch 7/1000
40675/40675 [==============================] - 0s 7u

I just the prediction to see if it is close to u_val.


In [75]:
    U=Y_test[10:]
    y_pred = model.predict(X_test[10:])
    print("Prediction Reseau 1 && U(x,t)")
    for i in range(0,5):
        print(str(y_pred[i])+" && " +str(U[i]))

    y_pred2 = model2.predict(Uxx[:10, :])
    print("Prediction Reseau 2 && Ut")
    for i in range(0,5):
        print(str(y_pred2[i])+" && " +str(Ut[i]))

Prediction Reseau 1 && U(x,t)
[-0.7784629] && [-0.69358093]
[-0.9636675] && [-0.99278892]
[-0.7711805] && [-0.73861599]
[0.2892991] && [0.16759193]
[0.21796387] && [0.22200285]
Prediction Reseau 2 && Ut
[0.2726933] && -0.004942828741348574
[-0.17720889] && -0.04682767790603827
[0.00755934] && -0.0004128849941225962
[0.22934471] && -0.0020660791386172286
[-0.23873019] && -0.002154885289421791
